In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [3]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [4]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])

In [6]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [7]:
SE6 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f: -c*(5*c+e)/(c+e)})

In [8]:
%time adm_SE6 = SE6.find_admissible_projectivities()

CPU times: user 329 ms, sys: 114 ms, total: 443 ms
Wall time: 1.08 s


In [9]:
len(adm_SE6)

48

In [10]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)

CPU times: user 32.1 ms, sys: 52.7 ms, total: 84.7 ms
Wall time: 119 ms


In [11]:
len(simm_SE6)

24

In [12]:
G_SE6 = Group(simm_SE6)

In [13]:
G_SE6.is_group()

True

In [14]:
G_SE6.get_order()

24

In [15]:
G_SE6.is_abelian()

False

In [16]:
G_SE6.get_divisors_of_order()

[1, 2, 3, 4, 6, 8, 12, 24]

In [17]:
G_SE6.get_order_of_elements()

[1, 2, 2, 4, 4, 3, 4, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 4, 3, 2, 4, 4, 3]

In [18]:
find_conditions_for_subfamilies(SE6, adm_SE6, simm_SE6)

[]

In [19]:
perms = []
for el in G_SE6:
    perms.append(Permutation(apply_proj_to_eck(el, SE6.eckardt_points)).to_cycles())

In [22]:
gp = PermutationGroup(perms)

In [23]:
gp.order()

24

In [24]:
gp.structure_description()

'S4'

In [48]:
nperms = [[cycle for cycle in perm if len(cycle)>1] for perm in perms]

In [51]:
for gen in gp.minimal_generating_set():
    print(nperms.index(gen.cycle_tuples()))
    print(gen)

13
(1,2,5)(3,6,4)
14
(1,6)(4,5)


In [58]:
gen1 = G_SE6[13]
gen2 = G_SE6[14]

In [66]:
[pl.labels for pl in SE6.tritangent_planes if pl.conditions == 0]

[['E1', 'G4', 'F14'],
 ['E2', 'G1', 'F12'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G4', 'F34'],
 ['E4', 'G2', 'F24'],
 ['F13', 'F24', 'F56']]

In [70]:
eck = SE6.eckardt_points
eck

[(0, 0, 0, 1),
 (0, -c^2 - c*e, 2*c^2 - 3*c*e - e^2, 0),
 (0, 1, 1, 0),
 (0, 0, 1, -1),
 (0, -c^2 - c*e, 2*c^2 - 3*c*e - e^2, -3*c^2 + 2*c*e + e^2),
 (0, c^2 + c*e, c^2 + c*e, -3*c^2 + 2*c*e + e^2)]

In [120]:
def get_dual_coordinates(pl):
    d = list(pl[0:3])
    m = [pl[5], -pl[4], pl[3]]
    return m+d

In [129]:
def get_planes(pl):
    #p01, p02, p03, p23, p31, p12
    dpl = get_dual_coordinates(pl)
    vrs = vector([x,y,z,t])
    eqns = [vector([0, -dpl[0], -dpl[1], -dpl[2]]), 
            vector([dpl[0], 0, dpl[5], -dpl[4]]), 
            vector([dpl[1], dpl[5], 0, -dpl[3]]), 
            vector([dpl[2], -dpl[4], dpl[3], 0])]
    planes = [eqn.dot_product(vrs) for eqn in eqns]
    return list(set([plane for plane in planes if plane != 0]))[0:2] 

In [123]:
new_lines = []
lines_plucker = [line.plucker for line in SE6.lines]
for i in range(len(eck)):
    point1 = eck[i]
    for j in range(i+1, len(eck)):
        point2 = eck[j]
        new_line_plucker = Point(matrix([point1, point2]).minors(2))
        if new_line_plucker not in lines_plucker and new_line_plucker not in [line.plucker for line in new_lines]:
            planes = get_planes(new_line_plucker)
            new_lines.append(Line(planes, [point1, point2], new_line_plucker))

In [124]:
new_lines

[(0, 0, 0, 0, c^2 + c*e, -2*c^2 + 3*c*e + e^2) [-x*c^2 - x*c*e, 2*y*c^2 + z*c^2 - 3*y*c*e + z*c*e - y*e^2],
 (0, 0, 0, 0, 1, 1) [-y + z, x],
 (0, 0, 0, -c^2 - c*e, c^2 + c*e, -2*c^2 + 3*c*e + e^2) [-x*c^2 - x*c*e, 2*y*c^2 + z*c^2 + t*c^2 - 3*y*c*e + z*c*e + t*c*e - y*e^2],
 (0, 0, 0, 1, -1, -1) [y - z - t, x]]

In [130]:
for line in new_lines:
    print(line.apply_proj(gen1) in new_lines, line.apply_proj(gen2) in new_lines)

True True
True True
True True
True True


In [81]:
for pl in new_lines:
    print(gen1*pl)

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 4 by 4 dense matrices over Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field' and 'Ambient free module of rank 6 over the integral domain Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field'

In [ ]:
permutation = []
for line in SE6.cl_lines.values():
    new_line = line.apply_proj(gen2)
    for key in SE6.cl_lines.keys():
        if SE6.cl_lines[key] == new_line:
            permutation.append(key)
            break

In [ ]:
permutation

['F13',
 'E2',
 'G2',
 'F34',
 'F35',
 'F36',
 'F12',
 'E3',
 'G3',
 'F24',
 'F25',
 'F26',
 'G1',
 'E1',
 'F56',
 'F46',
 'F45',
 'F23',
 'G4',
 'G5',
 'G6',
 'E4',
 'E5',
 'E6',
 'F16',
 'F15',
 'F14']